## 00. Getting Started

- __Activate virtual env (optional):__ To activate the virtual environment enter this your terminal:

```bash
      source openai-env/bin/activate
```





## 01. OpenAI Function Calling

![Open AI](https://i0.wp.com/bdtechtalks.com/wp-content/uploads/2024/01/openai-logo-abstract-background.jpg?ssl=1)

#### [Reference Docs](https://platform.openai.com/docs/guides/function-calling)

OpenAI **Function Calling** is a feature that allows developers to define **specific functions** that a language model can call during its interactions. This capability extends the model's functionality beyond **text generation** by enabling it to perform predefined **actions** or retrieve **information** from external sources. 

Developers set up these functions by specifying their **names**, **parameters**, and **descriptions**. During interactions, the model can use these functions to:

- **Query databases**
- **Execute business logic**
- **Retrieve real-time information**

This integration enhances the model's ability to handle complex scenarios and deliver more **tailored responses**, making it particularly useful for applications that require dynamic data retrieval or action execution within the conversation flow. Essentially, it enables language models to connect with external tools, thereby broadening their utility and effectiveness.

In [1]:
# Import necessary libraries
import os 
import openai 
from dotenv import load_dotenv

# Load environment variables form .env file
load_dotenv()

# Access environment variables and setup OpenAI client
openai_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_key

### Zero Shot Prompts

A **zero-shot prompt** is a type of prompt where you ask a language model to perform a task or answer a question without providing any specific examples or prior training for that particular task. The model is expected to generate a response based on its general understanding and pre-existing knowledge.

In [2]:
football_player = "Lionel Messi, widely regarded as one of the greatest footballers of all time, has amassed a record seven Ballon d'Or awards throughout his illustrious career and, at 37 years old, continues to be a prominent figure in the world of soccer."
football_player

"Lionel Messi, widely regarded as one of the greatest footballers of all time, has amassed a record seven Ballon d'Or awards throughout his illustrious career and, at 37 years old, continues to be a prominent figure in the world of soccer."

In [3]:
# A single prompt to extract information from "football_palyer" in a JSON format
prompt = f"""
Can you extract the following information from the given text and return it as a JSON object:

name
age
sports
awards

This is the body of text to extract the information from:
{football_player}
"""

In [4]:
prompt

"\nCan you extract the following information from the given text and return it as a JSON object:\n\nname\nage\nsports\nawards\n\nThis is the body of text to extract the information from:\nLionel Messi, widely regarded as one of the greatest footballers of all time, has amassed a record seven Ballon d'Or awards throughout his illustrious career and, at 37 years old, continues to be a prominent figure in the world of soccer.\n"

In [5]:
# Initiate OpenAI Client
from openai import OpenAI

client = OpenAI(api_key=openai_key)

In [6]:
# Prompt chat completion reponse
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

In [7]:
response

ChatCompletion(id='chatcmpl-9p9VhJFW1a7krpFkEbn1vX7rukCHr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "name": "Lionel Messi",\n  "age": 37,\n  "sports": "soccer",\n  "awards": 7\n}', role='assistant', function_call=None, tool_calls=None))], created=1721978313, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=35, prompt_tokens=101, total_tokens=136))

In [8]:
output=response.choices[0].message.content
output

'{\n  "name": "Lionel Messi",\n  "age": 37,\n  "sports": "soccer",\n  "awards": 7\n}'

Display **zero-shot prompt** response in JSON format

In [9]:
import json

json.loads(output)

{'name': 'Lionel Messi', 'age': 37, 'sports': 'soccer', 'awards': 7}

### Custom Function Calling

**Custom Function Calling** refers to the capability of integrating specific, user-defined functions into interactions with language models. This allows you to extend the functionality of the model by enabling it to invoke custom operations or processes based on the input it receives.

Breakdown of **`"name"`** parameter:

| **Attribute**  | **Description**                                                                                       |
|----------------|-------------------------------------------------------------------------------------------------------|
| **`"type"`**   | `"string"`: Specifies that the value for `"name"` should be a string, indicating it represents text. |
| **`"description"`** | `"Name of the player"`: Provides a brief explanation that this parameter refers to the player’s name. |
| **`"required"`** | `["name"]`: Indicates that the `"name"` field is mandatory in the output; the function must always include this information. |

Below **function call** snippet format is available in [Function Calling Example Section](https://platform.openai.com/docs/guides/function-calling)

In [10]:
custom_function = [
    {   
        "name": "extract_player_info",
        "description": "Get the player information from the body of input text",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the player",
                },
                "age": {
                    "type": "integer",
                    "description": "Age of the player",
                },
                "sports": {
                    "type": "string",
                    "description": "Sports played by the player",
                },
                "awards": {
                    
                    "type": "string",
                    "description": "Awards awarded to the player",
                },
            },
            # "required": ["name"],
        },
    },
]

In [11]:
# Prompt chat completion reponse using custom function calling
custom_func_call_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    functions=custom_function
)

In [12]:
custom_func_call_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"Lionel Messi","age":37,"sports":"football","awards":"seven Ballon d\'Or"}', name='extract_player_info'), tool_calls=None)

In [13]:
custom_output = custom_func_call_response.choices[0].message.function_call.arguments

In [14]:
json.loads(custom_output)

{'name': 'Lionel Messi',
 'age': 37,
 'sports': 'football',
 'awards': "seven Ballon d'Or"}

### Function Calls

Let's iterate over player information for **function calls**.

In [15]:
football_player2 = "Cristiano Ronaldo, widely regarded as one of the greatest footballers of all time, has amassed a record five Ballon d'Or awards throughout his illustrious career and, at 39 years old, continues to be a prominent figure in the world of soccer."

In [16]:
basketball_player = "Kobe Bryant, widely celebrated as one of the greatest basketball players of all time, won five NBA Championships with the Los Angeles Lakers and earned two NBA Finals MVP awards during his remarkable career. At 41 years old, Bryant remains a legendary figure in the world of basketball, renowned for his skill, work ethic, and competitive spirit."

In [17]:
players_info = [football_player, football_player2, basketball_player]

# Iterate players_info
for player in players_info:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": player
            }
        ],
        functions=custom_function,
        function_call="auto"
    )

    response = json.loads(response.choices[0].message.function_call.arguments)
    print(f"Player Info:\n {response}\n")

Player Info:
 {'name': 'Lionel Messi', 'age': 37, 'awards': "seven Ballon d'Or awards"}

Player Info:
 {'name': 'Cristiano Ronaldo', 'age': 39, 'awards': "five Ballon d'Or awards", 'sports': 'football'}

Player Info:
 {'name': 'Kobe Bryant', 'age': 41, 'sports': 'basketball', 'awards': 'five NBA Championships, two NBA Finals MVP awards'}



### Multiple Function Calls

In [18]:
custom_function_2 = [
    {   
        "name": "extract_player_info",
        "description": "Get the player information from the body of input text",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the player",
                },
                "awards": {
                    
                    "type": "string",
                    "description": "Awards awarded to the player",
                },
            },
            "required": ["name"],
        },
    },
]

In [19]:
custom_func = [custom_function, custom_function_2]
players_info = [football_player, football_player2, basketball_player]

# Iterate players_info
for idx, player in enumerate(players_info):
    for func in custom_func:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": player
                }
            ],
            functions=func,
            function_call="auto"
        )

        response = json.loads(response.choices[0].message.function_call.arguments)
        print(f"Player Info ({idx+1}):\n {response}\n")

Player Info (1):
 {'name': 'Lionel Messi', 'age': 37, 'awards': "seven Ballon d'Or"}

Player Info (1):
 {'name': 'Lionel Messi', 'awards': "seven Ballon d'Or awards"}

Player Info (2):
 {'name': 'Cristiano Ronaldo', 'age': 39, 'awards': "five Ballon d'Or awards", 'sports': 'soccer'}

Player Info (2):
 {'name': 'Cristiano Ronaldo', 'awards': "five Ballon d'Or awards"}

Player Info (3):
 {'name': 'Kobe Bryant', 'age': 41, 'sports': 'basketball', 'awards': 'five NBA Championships, two NBA Finals MVP awards'}

Player Info (3):
 {'name': 'Kobe Bryant', 'awards': 'Five NBA Championships, Two NBA Finals MVP awards'}



### Advanced Multiple Function Call

![Function Call](https://gradientflow.com/wp-content/uploads/2024/01/newsletter94-function-calling.jpeg)

Let's try out using another prompt that is used in **real-life scenario**.

In [ ]:
# Generate a response using prompt that depicts a real life scenario
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages =[
        {
            "role": "user",
            "content": "When's the next flight from Nepal to USA?"      # Promt
        }
    ]
)

In [24]:
# Display the response
response.choices[0].message.content

"I'm sorry, but as a language model AI, I am unable to provide real-time information or flight schedules. It's best to check with airlines or online booking platforms for the most up-to-date flight schedules and availability."

> **Note:** 
> - In industry applications, a response like <span style="color: green;">**_"I'm sorry, ... as an AI language model, I am unable to provide real-time information ..."_**</span> is not ideal. 
>
> - This limitation arises because OpenAI models are trained on static datasets and do not have access to real-time information or updates. 
>
> - Instead, modern industry-standard chatbots **leverage LLMs to understand the context of the conversation and integrate with third-party APIs or databases**. This approach enables them to provide up-to-date information and perform dynamic actions, showcasing the power and effectiveness of real-time interactions in practical applications.


In [27]:
flight_description_function = [
    {
        "name": "get_flight_info",
        "description": "Get flight details between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "Departure location, eg: NEP",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "Arrival location, eg: USA",
                },
            }
        },
        "required": ["loc_origin", "loc_destination"]
    }
]

In [28]:
# Prompt Example
user_prompt = "When's the next flight from Nepal to USA?"

In [30]:
advance_prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    # Add function calling
    functions=flight_description_function,
    # NOTE: This parameter set to "auto" indicated the model should automatically determine when to call a function based on the converstion's context
    function_call="auto"       
)

In [39]:
# Display the response
advance_prompt.choices[0].message.function_call.arguments

'{"loc_origin":"NEP","loc_destination":"USA"}'

> Here, the GPT Model has identified the properties that we need based on the prompt.

Now, lets **replicate the flight API** by making our own logic using function. If you want actual real-time 3rd aprty API, you can search up **Flight API** in [**RapidAPI**](https://rapidapi.com/).

In [50]:
from datetime import datetime, timedelta

def get_flight_info(loc_origin, loc_destination):
    """ Get flight information between two locations. """

    # Simulate API call
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=20)),
        "airline": "AirAsia",
        "flight_number": "AS123"
    }

    return json.dumps(flight_info)

In [40]:
# Pass response to dictionary
params = json.loads(advance_prompt.choices[0].message.function_call.arguments)
params

{'loc_origin': 'NEP', 'loc_destination': 'USA'}

In [45]:
origin = json.loads(advance_prompt.choices[0].message.function_call.arguments).get("loc_origin")
destination = json.loads(advance_prompt.choices[0].message.function_call.arguments).get("loc_destination")

origin, destination

('NEP', 'USA')

In [46]:
# Check `function_call` name
advance_prompt.choices[0].message.function_call.name

'get_flight_info'

In [53]:
flight_func = eval(advance_prompt.choices[0].message.function_call.name)

flight = flight_func(**params)
json.loads(flight)

{'loc_origin': 'NEP',
 'loc_destination': 'USA',
 'datetime': '2024-07-27 09:50:52.694837',
 'airline': 'AirAsia',
 'flight_number': 'AS123'}

Let's complete the **advanced function calling** and complete extraction of flight information

In [54]:
user_prompt

"When's the next flight from Nepal to USA?"

In [55]:
advance_prompt2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": user_prompt
        },
        {
            "role": "assistant",
            "name": advance_prompt.choices[0].message.function_call.name,
            "content": flight
        }
    ],
    functions=flight_description_function,
    function_call="auto"       
)

In [58]:
advance_prompt2.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The next flight from Nepal to USA is on July 27, 2024 at 09:50 AM. The flight is operated by AirAsia with the flight number AS123.', role='assistant', function_call=None, tool_calls=None))

In [59]:
advance_prompt2.choices[0].message.content

'The next flight from Nepal to USA is on July 27, 2024 at 09:50 AM. The flight is operated by AirAsia with the flight number AS123.'